In [30]:
import pandas as pd
import numpy as np
import faiss
import os

In [71]:
df = pd.read_pickle("./hnsw_all_cluster.pkl")

In [118]:
len(df)

10126

In [72]:
df.head(1)

,image_name,image_dir,results,image_features,cluster_id
0,g1327_czcqbl6.jpg,/nethome/kravicha3/aryan/project/dataset/Reddi...,"[1, 8887, 1415, 2028, 8885, 9459, 4633, 8590, ...","[[0.7295329, 0.30392087, 0.7336758, 0.41025618...",151


In [73]:
df['cluster_id']

0        151
1         13
2         69
3         69
4        154
        ... 
10121    105
10122     26
10123     61
10124     61
10125     61
Name: cluster_id, Length: 10126, dtype: int64

In [60]:
# features from the dataframe
df = pd.read_pickle('hnsw_all_image_results.pkl')
features = np.array(df['image_features'].to_list())
features = features.reshape(features.shape[0], features.shape[2])
# cluster training
ncentroids = 256
niter = 50
verbose = True
d = features.shape[1]
kmeans = faiss.Kmeans(d, ncentroids, niter=niter, verbose=verbose)
kmeans.train(features)


Clustering 10126 points in 2048D to 256 clusters, redo 1 times, 50 iterations
  Preprocessing in 0.01 s


331777.6875

### Get Label for Cluster ID

In [99]:
predictions_for_centroid = df[df['cluster_id'] == 4]
    
x = predictions_for_centroid['image_dir'].value_counts()
labels = []
for i in x.index:
    labels.append((os.path.split(i)[1], x[i]))

In [100]:
labels

[('_Sep_Blatter_after_a_protester_threw_money_at_him', 21),
 ('_Drawing_a_face_on_your_back_&amp;_reading_a_book', 1),
 ('_Kobe_Beef_Cube', 1)]

In [101]:
sum(x)

23

In [115]:
centroid_dataframe = pd.DataFrame(columns=["label", "num_correct", "num_total", "all_labels_list"])
for cluster_id in range(ncentroids):
    # maximum image_dir
    predictions_for_centroid = df[df['cluster_id'] == cluster_id]
    value_counts = predictions_for_centroid['image_dir'].value_counts()
    
    labels = list()
    for row_id in value_counts.index:
        labels.append((os.path.split(row_id)[1], value_counts[row_id]))
    
    centroid_dataframe.loc[len(centroid_dataframe.index)] = [labels[0][0], labels[0][1], sum(value_counts), labels]
    

In [116]:
centroid_dataframe

,label,num_correct,num_total,all_labels_list
0,_lunch_date_with_Lakers_girls_winner,1,1,"[(_lunch_date_with_Lakers_girls_winner, 1)]"
1,_Wombat_in_a_field,3,37,"[(_Wombat_in_a_field, 3), (_This_beautifully_l..."
2,_Tiger_standing_next_to_his_keeper,2,13,"[(_Tiger_standing_next_to_his_keeper, 2), (_Wo..."
3,_Cat_Sitting_at_a_Desk_in_School,4,8,"[(_Cat_Sitting_at_a_Desk_in_School, 4), (_My_f..."
4,_Sep_Blatter_after_a_protester_threw_money_at_him,21,23,[(_Sep_Blatter_after_a_protester_threw_money_a...
...,...,...,...,...
251,_Boy_in_his_cardboard_X_Wing,3,53,"[(_Boy_in_his_cardboard_X_Wing, 3), (_Airport_..."
252,_lunch_date_with_Lakers_girls_winner,37,37,"[(_lunch_date_with_Lakers_girls_winner, 37)]"
253,_A_man_standing_on_a_subway_vent,2,29,"[(_A_man_standing_on_a_subway_vent, 2), (_Cat_..."
254,_Kanye_West_caught_snoozing_at_the_baby_store,13,79,[(_Kanye_West_caught_snoozing_at_the_baby_stor...


In [117]:
sum(centroid_dataframe['num_total'])

10126

In [119]:
sum(centroid_dataframe['num_correct'])

3270

In [120]:
sum(centroid_dataframe['num_correct'])/sum(centroid_dataframe['num_total'])

0.3229310685364408

# best examples

In [124]:
centroid_dataframe['accuracy'] = centroid_dataframe['num_correct']/centroid_dataframe['num_total']

In [125]:
centroid_dataframe

,label,num_correct,num_total,all_labels_list,accuracy
0,_lunch_date_with_Lakers_girls_winner,1,1,"[(_lunch_date_with_Lakers_girls_winner, 1)]",1.000000
1,_Wombat_in_a_field,3,37,"[(_Wombat_in_a_field, 3), (_This_beautifully_l...",0.081081
2,_Tiger_standing_next_to_his_keeper,2,13,"[(_Tiger_standing_next_to_his_keeper, 2), (_Wo...",0.153846
3,_Cat_Sitting_at_a_Desk_in_School,4,8,"[(_Cat_Sitting_at_a_Desk_in_School, 4), (_My_f...",0.500000
4,_Sep_Blatter_after_a_protester_threw_money_at_him,21,23,[(_Sep_Blatter_after_a_protester_threw_money_a...,0.913043
...,...,...,...,...,...
251,_Boy_in_his_cardboard_X_Wing,3,53,"[(_Boy_in_his_cardboard_X_Wing, 3), (_Airport_...",0.056604
252,_lunch_date_with_Lakers_girls_winner,37,37,"[(_lunch_date_with_Lakers_girls_winner, 37)]",1.000000
253,_A_man_standing_on_a_subway_vent,2,29,"[(_A_man_standing_on_a_subway_vent, 2), (_Cat_...",0.068966
254,_Kanye_West_caught_snoozing_at_the_baby_store,13,79,[(_Kanye_West_caught_snoozing_at_the_baby_stor...,0.164557


In [127]:
sorted_df = centroid_dataframe.sort_values(by='accuracy', ascending=False)

In [133]:
sorted_df[sorted_df['num_total']> 10].head(15)

,label,num_correct,num_total,all_labels_list,accuracy
67,_This_rainbow_woman_on_the_beach,25,25,"[(_This_rainbow_woman_on_the_beach, 25)]",1.000000
178,_News_interns_running_out_with_Supreme_Court_r...,33,33,[(_News_interns_running_out_with_Supreme_Court...,1.000000
187,_Jeremy_Clarkson_laughing_at_Chris_Evans,26,26,"[(_Jeremy_Clarkson_laughing_at_Chris_Evans, 26)]",1.000000
83,_Swedens_prime_minister_is_sad_and_left_out_wh...,30,30,[(_Swedens_prime_minister_is_sad_and_left_out_...,1.000000
40,Someone_suggested_I_post_this_here_Gf_sent_me_...,15,15,[(Someone_suggested_I_post_this_here_Gf_sent_m...,1.000000
201,_Ridiculously_photogenic_volleyball_player,19,19,"[(_Ridiculously_photogenic_volleyball_player, ...",1.000000
20,_An_angry_dog_standing_upright,18,18,"[(_An_angry_dog_standing_upright, 18)]",1.000000
147,_Canadian_PM_Justin_Trudeau_hesitating_to_shak...,60,60,[(_Canadian_PM_Justin_Trudeau_hesitating_to_sh...,1.000000
60,_Alexis_Ohanian_(CEO_and_founder_of_reddit)_ho...,23,23,[(_Alexis_Ohanian_(CEO_and_founder_of_reddit)_...,1.000000
250,_This_picture_of_a_man_with_a_beard_looking_st...,29,29,[(_This_picture_of_a_man_with_a_beard_looking_...,1.000000


In [132]:
sorted_df[sorted_df['num_total']> 30].head(15)

,label,num_correct,num_total,all_labels_list,accuracy
178,_News_interns_running_out_with_Supreme_Court_r...,33,33,[(_News_interns_running_out_with_Supreme_Court...,1.000000
147,_Canadian_PM_Justin_Trudeau_hesitating_to_shak...,60,60,[(_Canadian_PM_Justin_Trudeau_hesitating_to_sh...,1.000000
252,_lunch_date_with_Lakers_girls_winner,37,37,"[(_lunch_date_with_Lakers_girls_winner, 37)]",1.000000
245,_Emilia_Clarke's_reaction_to_Kristen_Wiig_play...,35,36,[(_Emilia_Clarke's_reaction_to_Kristen_Wiig_pl...,0.972222
220,Kid_running_from_guy_in_mankini,39,41,"[(Kid_running_from_guy_in_mankini, 39), (_Ewan...",0.951220
153,_The_Supreme_Leader_wears_a_fedora_and_gives_a...,34,36,[(_The_Supreme_Leader_wears_a_fedora_and_gives...,0.944444
100,_Sad_old_man_wearing_a_cap_with_obscene_langua...,31,33,[(_Sad_old_man_wearing_a_cap_with_obscene_lang...,0.939394
68,_These_squirrels_fighting,41,44,"[(_These_squirrels_fighting, 41), (_Wombat_in_...",0.931818
41,_Penguin_posing_in_front_of_a_ship,34,38,"[(_Penguin_posing_in_front_of_a_ship, 34), (_A...",0.894737
235,_North_Korea's_New_Untouched_Photo_of_Kim_Jong-Un,31,35,[(_North_Korea's_New_Untouched_Photo_of_Kim_Jo...,0.885714


In [134]:
sorted_df[sorted_df['num_total']> 50].head(15)

,label,num_correct,num_total,all_labels_list,accuracy
147,_Canadian_PM_Justin_Trudeau_hesitating_to_shak...,60,60,[(_Canadian_PM_Justin_Trudeau_hesitating_to_sh...,1.000000
214,_A_pawn_looking_in_the_mirror,45,52,"[(_A_pawn_looking_in_the_mirror, 45), (_A_dehy...",0.865385
160,_You_Catch_Your_Roommate_With_His_Pants_Down,43,52,[(_You_Catch_Your_Roommate_With_His_Pants_Down...,0.826923
145,_A_selfie_from_the_driver_of_the_Pope,39,54,"[(_A_selfie_from_the_driver_of_the_Pope, 39), ...",0.722222
130,_This_teacher_from_my_Facebook_feed,43,67,"[(_This_teacher_from_my_Facebook_feed, 43), (_...",0.641791
231,_Streaker_stealing_the_show_at_a_Tennis_game,36,60,[(_Streaker_stealing_the_show_at_a_Tennis_game...,0.600000
70,_Two_People_Looking_At_Nice_View,32,54,"[(_Two_People_Looking_At_Nice_View, 32), (_Gir...",0.592593
142,_Trump_trying_to_close_his_pen,44,82,"[(_Trump_trying_to_close_his_pen, 44), (_Adam_...",0.536585
110,_Bernie_Sanders_browsing_reddit,43,93,"[(_Bernie_Sanders_browsing_reddit, 43), (_A_ba...",0.462366
196,_Tiger_standing_next_to_his_keeper,34,74,"[(_Tiger_standing_next_to_his_keeper, 34), (_D...",0.459459


### Getting nearest points to cluster centroid

In [61]:
index_cluster = faiss.IndexFlatL2 (d)
index_cluster.add(features)
D, I = index_cluster.search(kmeans.centroids, 100)

In [62]:
I.shape

(256, 100)

In [63]:
centroid_labels = []
for centroid_id in range(I.shape[0]):
    label = {}
    for label_id in I[centroid_id]:
        thread_dir = df.iloc[label_id]['image_dir']
        thread_dir = os.path.split(thread_dir)[1]
        if thread_dir in label:
            label[thread_dir] += 1
        else:
            label[thread_dir] = 1
    centroid_labels.append(label)

In [64]:
len(centroid_labels)

256

In [65]:
centroid_num_threads = []
for centroid_id in range(len(centroid_labels)):
    max_n = 0
    max_l = ""
    for i in centroid_labels[centroid_id]:
        num = centroid_labels[centroid_id][i]
        if num>max_n:
            max_n = num
            max_l = i
    centroid_num_threads.append((max_n, max_l))

In [66]:
centroid_num_threads

[(27, '_Streaker_stealing_the_show_at_a_Tennis_game'),
 (15, '_You_Catch_Your_Roommate_With_His_Pants_Down'),
 (14, '_Headless_hockey_player_and_terrified_referee'),
 (17, '_Cat_Sitting_at_a_Desk_in_School'),
 (25, '_Sep_Blatter_after_a_protester_threw_money_at_him'),
 (34, '_Foggy_day'),
 (40, '_These_squirrels_fighting'),
 (9, '_Headless_hockey_player_and_terrified_referee'),
 (21, '_Dog_wearing_a_mask_at_a_dino_park'),
 (13, '_Weightlifter_jumping_up_after_a_successful_lift'),
 (20, '_Man_driving_his_Tricer-a-car'),
 (17, '_Group_of_Shaved_Llamas'),
 (29, '_These_piles_of_spice_at_a_market_in_Marrakesh'),
 (23, '_LeBron_James_holding_a_trophy'),
 (29, '_Man_observing_an_oil_painting_of_himself'),
 (22, '_Headless_hockey_player_and_terrified_referee'),
 (15, '_Headless_hockey_player_and_terrified_referee'),
 (7, '_This_teacher_from_my_Facebook_feed'),
 (19, '_Palestinian_man_taking_selfie_while_Israeli_police_chase_him'),
 (17, '_Adam_Savage_after_his_morning_workout'),
 (22, '_This_

In [67]:
temp = set()
for i in centroid_num_threads:
    temp.add(i[1])
len(temp)

108

### Recall